## Test for pytorch

#### Import necessary libraries

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from DeepEverest import DeepEverest

#### Load data for the demonstration (MNIST)

In [2]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=1000, shuffle=False)

#### Construct a model class

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )
    def forward(self, inputs):
        return self.model(inputs)

#### Prepare datasets, and construct DeepEverest class

In [4]:
dataset = torch.tensor([])
dataset_labels = torch.tensor([])
for num, (data, data_set) in enumerate(test_loader):
    if num == 0:
        dataset = torch.tensor(data)
        dataset_labels = torch.tensor(data_set)
    else:
        dataset = torch.cat((dataset, data), 0)
        dataset_labels = torch.cat((dataset_labels, data_set), 0)
model = Net()
lib_file = "index/build/lib.linux-x86_64-3.8/deepeverst_index.cpython-38-x86_64-linux-gnu.so"
model = DeepEverest(model, True, lib_file, dataset)

/tmp/ipykernel_10398/2494629665.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = torch.tensor(data)
/tmp/ipykernel_10398/2494629665.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_labels = torch.tensor(data_set)


#### Construct a new pytorch network, and construct the network into the DeepEverest API class

In [5]:
model = Net()
model = DeepEverest(model, True, lib_file, dataset)

#### Load a pretrained weight as an example

In [6]:
model.load_weights('mnist.pth')

#### print out all the layer outputs

In [7]:
model.get_layer_outputs()

{'': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc550>,
 'model': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc6a0>,
 'model.0': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc2b0>,
 'model.1': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc790>,
 'model.2': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc7f0>,
 'model.3': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc910>,
 'model.4': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc5e0>,
 'model.5': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc670>,
 'model.6': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc730>,
 'model.7': <torch.utils.hooks.RemovableHandle at 0x7f09eaedca30>,
 'model.8': <torch.utils.hooks.RemovableHandle at 0x7f09eaedc0a0>,
 'model.9': <torch.utils.hooks.RemovableHandle at 0x7f0a4b4a0070>,
 'model.10': <torch.utils.hooks.RemovableHandle at 0x7f0a4b4a0100>,
 'model.11': <torch.utils.hooks.RemovableHandle at 0x7f0a4b4a01f0>,
 'model.12': <torch.utils.hooks.RemovableHandle at 0x7f09e868b3a0>}

In [8]:
dataset_labels

tensor([7, 2, 1,  ..., 4, 5, 6])

In [9]:
for name, module in model.get_model().named_modules():
    print(name)


model
model.0
model.1
model.2
model.3
model.4
model.5
model.6
model.7
model.8
model.9
model.10
model.11
model.12


#### Choose one of the layer names

In [10]:
all_layer_names = model.get_all_layer_names()
print(all_layer_names)
layer_name = "model.2"
layer_id = all_layer_names.index(layer_name)
print(layer_id)

['', 'model', 'model.0', 'model.1', 'model.2', 'model.3', 'model.4', 'model.5', 'model.6', 'model.7', 'model.8', 'model.9', 'model.10', 'model.11', 'model.12']
4


#### The following two lines are demonstration purpose, this is not needed in use of DeepEverest

In [11]:
layer_result = model.get_layer_results_by_layer_id(layer_id)

In [12]:
print(layer_result.shape)

(10000, 10, 12, 12)


#### Give top-k activation neurons of image 659 as an example

In [13]:
topk_activations = model.get_topk_activations_given_images([659], layer_name, 20)
print(topk_activations)

[[(15.056664, (4, 4, 6)), (14.274166, (4, 5, 6)), (13.664486, (4, 6, 6)), (13.554054, (4, 3, 6)), (13.502932, (4, 8, 6)), (13.477571, (4, 9, 6)), (13.340709, (1, 11, 6)), (13.27995, (4, 4, 7)), (13.197538, (1, 5, 3)), (13.188957, (4, 3, 7)), (13.090613, (1, 10, 6)), (12.774042, (1, 5, 4)), (12.733601, (4, 2, 6)), (12.522223, (4, 7, 6)), (12.089676, (4, 5, 7)), (11.97962, (1, 11, 7)), (11.603155, (4, 8, 5)), (11.496715, (4, 2, 7)), (11.474282, (0, 3, 7)), (11.457584, (1, 4, 4))]]


#### Verify the results are correct (same) by brute force. This is for demonstration only, no need when using DeepEverest

In [14]:
res = []
image_samples = []
for image_sample_id in [659]:
    image_samples.append(dataset[image_sample_id])
image_samples = torch.stack(image_samples)
layer_result_image_samples = model.get_layer_result_by_layer_name(image_samples, layer_name)
brute_force = []
for i in range(layer_result_image_samples.shape[1]):
    for j in range(layer_result_image_samples.shape[2]):
        for k in range(layer_result_image_samples.shape[3]):
            brute_force.append((layer_result_image_samples[0, i, j, k], (i, j, k)))
sorted_array = sorted(brute_force, key=lambda k:k[0], reverse=True)
print(sorted_array[:20])

[(tensor(15.0567), (4, 4, 6)), (tensor(14.2742), (4, 5, 6)), (tensor(13.6645), (4, 6, 6)), (tensor(13.5541), (4, 3, 6)), (tensor(13.5029), (4, 8, 6)), (tensor(13.4776), (4, 9, 6)), (tensor(13.3407), (1, 11, 6)), (tensor(13.2800), (4, 4, 7)), (tensor(13.1975), (1, 5, 3)), (tensor(13.1890), (4, 3, 7)), (tensor(13.0906), (1, 10, 6)), (tensor(12.7740), (1, 5, 4)), (tensor(12.7336), (4, 2, 6)), (tensor(12.5222), (4, 7, 6)), (tensor(12.0897), (4, 5, 7)), (tensor(11.9796), (1, 11, 7)), (tensor(11.6032), (4, 8, 5)), (tensor(11.4967), (4, 2, 7)), (tensor(11.4743), (0, 3, 7)), (tensor(11.4576), (1, 4, 4))]


#### Construct Neuron groups (Choose the top-3 neurons for demonstration)

In [15]:
from NeuronGroup import *
image_sample_id = 659
topk_activations_neurons = [x[1] for x in topk_activations[0]]
print(topk_activations_neurons)
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=topk_activations_neurons[:3])

[(4, 4, 6), (4, 5, 6), (4, 6, 6), (4, 3, 6), (4, 8, 6), (4, 9, 6), (1, 11, 6), (4, 4, 7), (1, 5, 3), (4, 3, 7), (1, 10, 6), (1, 5, 4), (4, 2, 6), (4, 7, 6), (4, 5, 7), (1, 11, 7), (4, 8, 5), (4, 2, 7), (0, 3, 7), (1, 4, 4)]


#### Query the top 20 closest images with respect to the top 3 neurons found in the last step

In [16]:
top_k, exit_msg = model.answer_query_with_guarantee(layer_id, image_sample_id, 20, neuron_group)

image 659, size of neuron group 3
threshold: 0.10146905481815338, max in answer: 1.40303635597229, images run: 67
threshold: 0.20118562877178192, max in answer: 1.1438370943069458, images run: 132
threshold: 0.2788737714290619, max in answer: 0.9198803901672363, images run: 196
threshold: 0.42958271503448486, max in answer: 0.7807649374008179, images run: 260
threshold: 0.5854843854904175, max in answer: 0.7519282102584839, images run: 325
threshold: 0.7778107523918152, max in answer: 0.7440550923347473, images run: 389
======================= NTA exited =======================


#### Print out distance and corresponding image indices, the distance are the absolute value of the first element, and the image id are the second element

In [17]:
print("The query result (distance from the input image, image ids) array is:")
print(top_k)
print(exit_msg)

The query result (distance from the input image, image ids) array is:
[(-0.7440551, 2915), (-0.7409354, 6074), (-0.6970588, 8109), (-0.6478751, 7822), (-0.6055464, 3611), (-0.58405477, 6118), (-0.5718313, 4011), (-0.53731173, 8100), (-0.52412087, 4097), (-0.5215469, 4861), (-0.46935377, 6988), (-0.44762346, 6096), (-0.4275496, 8048), (-0.4268946, 7783), (-0.42228025, 1909), (-0.41333342, 3229), (-0.3712188, 1480), (-0.23766346, 8679), (-0.113409005, 8112), (0.0, 659)]
termination: images run: 389


#### Verify the results are correct (same) by brute force. This is for demonstration only, no need when using DeepEverest

In [18]:
layer_result = model.get_layer_results_by_layer_id(layer_id)
print(layer_result.shape)
print(topk_activations_neurons[:3])
search_target = np.array([layer_result[659][(4, 4, 6)], layer_result[659][(4, 5, 6)], layer_result[659][(4, 6, 6)]])
print(search_target)
result_array = []
for i in range(len(dataset)):
    number = np.array([layer_result[i][(4, 4, 6)], layer_result[i][(4, 5, 6)], layer_result[i][(4, 6, 6)]])
    distance = np.sqrt(np.sum((search_target - number) * (search_target - number)))
    result_array.append((distance, i))
result_array = sorted(result_array, key=lambda k: k[0], reverse=True)
print()
print("The query result (distance from the input image, image ids) array by brute force is:")
print(result_array[-20:])

(10000, 10, 12, 12)
[(4, 4, 6), (4, 5, 6), (4, 6, 6)]
[15.056664 14.274166 13.664486]

The query result (distance from the input image, image ids) array by brute force is:
[(0.7440551, 2915), (0.7409354, 6074), (0.6970588, 8109), (0.6478751, 7822), (0.6055464, 3611), (0.58405477, 6118), (0.5718313, 4011), (0.53731173, 8100), (0.52412087, 4097), (0.5215469, 4861), (0.46935377, 6988), (0.44762346, 6096), (0.4275496, 8048), (0.4268946, 7783), (0.42228025, 1909), (0.41333342, 3229), (0.3712188, 1480), (0.23766346, 8679), (0.113409005, 8112), (0.0, 659)]


In [19]:
image_ids = [1, 0, 321, 810]
image_ids

[1, 0, 321, 810]

#### This tool is capable of search group of images in one query

In [20]:
topk_activations = model.get_topk_activations_given_images(image_ids, layer_name, 20)

In [21]:
topk_activations

[[(15.404506, (1, 10, 6)),
  (14.740844, (1, 10, 9)),
  (14.725686, (1, 10, 5)),
  (14.658377, (1, 10, 4)),
  (14.074226, (1, 10, 7)),
  (14.018891, (1, 10, 10)),
  (13.745972, (1, 10, 3)),
  (13.607746, (1, 2, 5)),
  (13.291188, (1, 10, 8)),
  (12.143644, (1, 2, 4)),
  (11.776539, (4, 2, 6)),
  (11.771938, (4, 6, 4)),
  (11.722886, (4, 8, 3)),
  (11.716561, (1, 1, 5)),
  (11.585614, (4, 3, 6)),
  (11.531162, (4, 7, 3)),
  (11.222839, (4, 5, 4)),
  (11.1286335, (4, 4, 5)),
  (11.101066, (1, 10, 11)),
  (11.002071, (4, 5, 5))],
 [(14.003622, (1, 4, 4)),
  (13.753479, (1, 4, 5)),
  (13.233683, (1, 4, 3)),
  (13.210785, (1, 4, 6)),
  (12.756104, (1, 4, 7)),
  (10.772835, (4, 11, 4)),
  (10.763884, (4, 10, 5)),
  (10.652996, (4, 4, 8)),
  (10.51455, (1, 3, 3)),
  (10.362712, (1, 4, 2)),
  (10.331228, (1, 3, 2)),
  (9.836474, (4, 9, 5)),
  (9.813205, (4, 5, 8)),
  (9.546487, (4, 10, 4)),
  (9.501915, (4, 8, 6)),
  (9.265276, (4, 11, 5)),
  (9.024993, (4, 7, 7)),
  (8.871638, (4, 6, 7)),
  (

#### A group of neurons can be specified by the user to do the query

In [22]:
image_sample_id = 810
neuron_list = [(1, 4, 4), (1, 4, 3), (1, 4, 5)]
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=neuron_list)

#### Find the closest image with respect to the neuron group specified

In [23]:
top_k, exit_msg = model.answer_query_with_guarantee(layer_id, image_sample_id, 20, neuron_group)
print(top_k)

image 810, size of neuron group 3
threshold: 0.4012274742126465, max in answer: 1.244203805923462, images run: 67
threshold: 0.7033581137657166, max in answer: 1.01547372341156, images run: 131
threshold: 0.9670814871788025, max in answer: 0.9958177804946899, images run: 195
threshold: 1.2530744075775146, max in answer: 0.9958177804946899, images run: 260
======================= NTA exited =======================
[(-0.9958178, 9990), (-0.9594661, 8555), (-0.9563361, 6316), (-0.92077124, 6050), (-0.88487566, 6158), (-0.86679006, 5179), (-0.85618293, 3743), (-0.84050524, 60), (-0.83489597, 1935), (-0.81305367, 7960), (-0.78954655, 2187), (-0.73703915, 6366), (-0.6998202, 5326), (-0.6811, 4865), (-0.6809903, 6006), (-0.65254635, 4064), (-0.6416337, 7650), (-0.6308938, 3125), (-0.57279134, 822), (0.0, 810)]


#### Another example

In [24]:
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=[(5, 0, 2), (8, 2, 2), (2, 4, 1)])

In [25]:
top_k, exit_msg = model.answer_query_with_guarantee(layer_id, image_sample_id, 20, neuron_group)
print(top_k)

image 810, size of neuron group 3
threshold: 0.03818821907043457, max in answer: 0.1384831666946411, images run: 427
threshold: 0.20468735694885254, max in answer: 0.07035708427429199, images run: 851
======================= NTA exited =======================
[(-0.070357084, 4176), (-0.0684191, 4659), (-0.06708133, 3394), (-0.06519842, 9109), (-0.05986345, 2700), (-0.057710648, 554), (-0.052951217, 4042), (-0.045683503, 1100), (-0.04501462, 6053), (-0.04026103, 9618), (-0.039102912, 4421), (-0.03840089, 738), (-0.03818822, 812), (-0.03238964, 3431), (-0.026446223, 362), (-0.025786996, 924), (-0.024932742, 6871), (-0.020506978, 6075), (-0.0010017157, 2115), (0.0, 810)]


#### Can construct indices for multiple layers, and the index constructed so far can be saved to disk

In [26]:
model.save_index_map("saved_map")

#### Saved indices can be reload back

In [27]:
model.load_index_map("saved_map")

## Test for tensorflow

#### This part is aimed to show that the code also works with Tensorflow model, the process is similar, and user should create a super class shown in MnistVGG_api class, and use this class to do queries. The procedure is very similar comparing to using a Pytorch model. 

In [28]:
import os
import numpy as np
from utils import load_mnist_vgg_dataset_model
from models.MnistVGG_api import MnistVGG

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [29]:
x_train, y_train, x_test, y_test = load_mnist_vgg_dataset_model()
dataset = x_test
dataset_labels = y_test
del x_train
del y_train
model = MnistVGG(lib_file, dataset)
all_layer_names = model.get_all_layer_names()
print(all_layer_names)

2022-07-26 14:44:29.950431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 14:44:29.952892: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 14:44:29.953203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 14:44:29.953640: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

['input_1', 'zero_padding2d', 'conv2d', 'batch_normalization', 'activation', 'conv2d_1', 'batch_normalization_1', 'activation_1', 'max_pooling2d', 'conv2d_2', 'batch_normalization_2', 'activation_2', 'conv2d_3', 'batch_normalization_3', 'activation_3', 'max_pooling2d_1', 'conv2d_4', 'batch_normalization_4', 'activation_4', 'conv2d_5', 'batch_normalization_5', 'activation_5', 'conv2d_6', 'batch_normalization_6', 'activation_6', 'max_pooling2d_2', 'conv2d_7', 'batch_normalization_7', 'activation_7', 'conv2d_8', 'batch_normalization_8', 'activation_8', 'conv2d_9', 'batch_normalization_9', 'activation_9', 'max_pooling2d_3', 'conv2d_10', 'batch_normalization_10', 'activation_10', 'conv2d_11', 'batch_normalization_11', 'activation_11', 'conv2d_12', 'batch_normalization_12', 'activation_12', 'max_pooling2d_4', 'flatten', 'dense', 'dropout', 'dense_1', 'dropout_1', 'dense_2', 'activation_13']


In [30]:
layer_name = "activation_12"
layer_id = all_layer_names.index(layer_name)

In [31]:
print(dataset.shape)
print(dataset_labels.shape)

(10000, 28, 28, 1)
(10000, 10)


#### For demonstration only, no need for using DeepEverest

In [32]:
layer_result = model.get_layer_results_by_layer_id(layer_id)
print(layer_result.shape)
print(type(layer_result))

2022-07-26 14:44:31.708332: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-07-26 14:44:32.424525: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


(10000, 2, 2, 512)
<class 'numpy.ndarray'>


In [33]:
topk_activations = model.get_topk_activations_given_images([659], layer_name, 20)
print(topk_activations)

[[(2.2470694, (1, 1, 374)), (1.9390907, (0, 0, 113)), (1.9011351, (0, 0, 358)), (1.845798, (1, 0, 335)), (1.824193, (1, 1, 62)), (1.7726831, (1, 0, 182)), (1.7536618, (1, 1, 129)), (1.7496979, (0, 0, 95)), (1.7491616, (1, 0, 441)), (1.6753764, (1, 1, 177)), (1.6667678, (0, 0, 46)), (1.6587296, (0, 1, 340)), (1.6440464, (0, 1, 285)), (1.6398698, (1, 1, 428)), (1.6205434, (0, 1, 365)), (1.6141273, (0, 1, 461)), (1.5758126, (0, 0, 245)), (1.5589455, (0, 0, 45)), (1.5521427, (0, 1, 336)), (1.5128995, (0, 1, 379))]]


#### Verify the results are correct (same) by brute force. This is for demonstration only, no need when using DeepEverest

In [34]:
res = []
image_samples = []
for image_sample_id in [659]:
    image_samples.append(dataset[image_sample_id])
layer_result_image_samples = model.get_layer_result_by_layer_name(image_samples, layer_name)
brute_force = []
for i in range(layer_result_image_samples.shape[1]):
    for j in range(layer_result_image_samples.shape[2]):
        for k in range(layer_result_image_samples.shape[3]):
            brute_force.append((layer_result_image_samples[0, i, j, k], (i, j, k)))
sorted_array = sorted(brute_force, key=lambda k:k[0], reverse=True)
print(sorted_array[:20])

[(2.2470694, (1, 1, 374)), (1.9390907, (0, 0, 113)), (1.9011351, (0, 0, 358)), (1.845798, (1, 0, 335)), (1.824193, (1, 1, 62)), (1.7726831, (1, 0, 182)), (1.7536618, (1, 1, 129)), (1.7496979, (0, 0, 95)), (1.7491616, (1, 0, 441)), (1.6753764, (1, 1, 177)), (1.6667678, (0, 0, 46)), (1.6587296, (0, 1, 340)), (1.6440464, (0, 1, 285)), (1.6398698, (1, 1, 428)), (1.6205434, (0, 1, 365)), (1.6141273, (0, 1, 461)), (1.5758126, (0, 0, 245)), (1.5589455, (0, 0, 45)), (1.5521427, (0, 1, 336)), (1.5128995, (0, 1, 379))]


#### Choose top-3 neurons for image search

In [35]:
from NeuronGroup import *
image_sample_id = 659
topk_activations_neurons = [x[1] for x in topk_activations[0]]
neuron_group = NeuronGroup(model.get_model(), layer_id, neuron_idx_list=topk_activations_neurons[:3])
print(topk_activations_neurons[:3])

[(1, 1, 374), (0, 0, 113), (0, 0, 358)]


In [36]:
top_k, exit_msg = model.answer_query_with_guarantee(layer_id, image_sample_id, 20, neuron_group)

image 659, size of neuron group 3
threshold: 0.009087341837584972, max in answer: 1.0652905702590942, images run: 446
threshold: 0.23611624538898468, max in answer: 0.7390373945236206, images run: 879
threshold: 0.2542610764503479, max in answer: 0.7369700074195862, images run: 1320
threshold: 0.466909259557724, max in answer: 0.6489895582199097, images run: 1707
threshold: 0.5761297345161438, max in answer: 0.6489895582199097, images run: 2125
threshold: 0.741787850856781, max in answer: 0.6489895582199097, images run: 2537
======================= NTA exited =======================


In [37]:
print("The query result (distance from the input image, image ids) array is:")
print(top_k)
print(exit_msg)

The query result (distance from the input image, image ids) array is:
[(-0.64898956, 8043), (-0.6488446, 3073), (-0.6324043, 2915), (-0.63099045, 9505), (-0.6308887, 7985), (-0.61490345, 9664), (-0.57324046, 1941), (-0.570809, 9637), (-0.5610432, 1903), (-0.55843884, 3166), (-0.5519713, 468), (-0.53630835, 5649), (-0.5042943, 667), (-0.37808788, 7902), (-0.3767796, 3055), (-0.37318093, 5655), (-0.3371253, 4176), (-0.32670704, 4199), (-0.15818997, 3808), (0.0, 659)]
termination: images run: 2537


#### Verify the results are correct (same) by brute force. This is for demonstration only, no need when using DeepEverest

In [38]:
print(layer_result.shape)
print(topk_activations_neurons[:3])
search_target = np.array([layer_result[659][(1, 1, 374)], layer_result[659][(0, 0, 113)], layer_result[659][(0, 0, 358)]])
print(search_target)
result_array = []
for i in range(len(dataset)):
    number = np.array([layer_result[i][(1, 1, 374)], layer_result[i][(0, 0, 113)], layer_result[i][(0, 0, 358)]])
    distance = np.sqrt(np.sum((search_target - number) * (search_target - number)))
    result_array.append((distance, i))
result_array = sorted(result_array, key=lambda k: k[0], reverse=True)
print()
print("The query result (distance from the input image, image ids) array by brute force is:")
print(result_array[-20:])

(10000, 2, 2, 512)
[(1, 1, 374), (0, 0, 113), (0, 0, 358)]
[2.247072  1.9390905 1.9011344]

The query result (distance from the input image, image ids) array by brute force is:
[(0.6489887, 8043), (0.64884716, 3073), (0.6324064, 2915), (0.6309891, 9505), (0.6308879, 7985), (0.6149021, 9664), (0.5732398, 1941), (0.5708099, 9637), (0.5610427, 1903), (0.5584397, 3166), (0.5519719, 468), (0.53630936, 5649), (0.50429475, 667), (0.3780878, 7902), (0.37678048, 3055), (0.37317818, 5655), (0.337125, 4176), (0.32670948, 4199), (0.15818849, 3808), (0.0, 659)]


In [39]:
layer_name = "activation_10"
layer_id = all_layer_names.index(layer_name)

In [40]:
top_k, exit_msg = model.answer_query_with_guarantee(layer_id, image_sample_id, 20, neuron_group)

image 659, size of neuron group 3
threshold: 3.310506820678711, max in answer: 1.5521913766860962, images run: 443
======================= NTA exited =======================


In [41]:
print(top_k)
print(exit_msg)

[(-1.5521914, 5831), (-1.5382272, 6105), (-1.5171522, 1156), (-1.473711, 7844), (-1.4282893, 5887), (-1.2838352, 2434), (-1.2651086, 1039), (-1.250756, 689), (-1.2400525, 9839), (-1.1397568, 726), (-1.1281551, 1173), (-1.1208655, 3213), (-1.1054935, 730), (-1.0767243, 746), (-1.0691105, 6188), (-1.0641898, 1581), (-0.98903173, 716), (-0.9283188, 4966), (-0.8697332, 649), (0.0, 659)]
termination: images run: 443
